In [1]:
import os
import json
import torch.multiprocessing as mp
from PIL import Image

# From YOLO to COCO annotation

In [2]:
# CONFIG
repo_dir = os.getcwd().split('dslab25')[0] + 'dslab25/'
root_dir = repo_dir + "training/vacuum_pump"
image_dir = os.path.join(root_dir, "images/augmented")
label_dir = os.path.join(root_dir, "annotation/augmented")
coco_path = os.path.join(root_dir, "coco_annotations.json")

# Collect category mapping
stage_folders = [f"stage_{i}" for i in range(8)]
category_mapping = {name: i for i, name in enumerate(stage_folders)}  # name -> ID
categories = [{"id": i, "name": name} for name, i in category_mapping.items()]

# Initialize COCO structure
coco_output = {
	"images": [],
	"annotations": [],
	"categories": categories
}

image_id = 0
annotation_id = 0

# Traverse through each stage folder
for class_folder in stage_folders:
	img_folder = os.path.join(image_dir, class_folder)
	label_folder = os.path.join(label_dir, class_folder)
	category_id = category_mapping[class_folder]

	for filename in os.listdir(img_folder):
		if not filename.endswith(".jpg"):
			continue

		image_path = os.path.join(img_folder, filename)
		label_path = os.path.join(label_folder, filename.replace(".jpg", ".txt"))

		# Read image size
		with Image.open(image_path) as img:
			width, height = img.size

		# Add image entry
		coco_output["images"].append({
			"id": image_id,
			"file_name": f"{class_folder}/{filename}",
			"width": width,
			"height": height
		})

		# Process annotation
		if os.path.exists(label_path):
			with open(label_path, "r") as f:
				for line in f:
					parts = line.strip().split()
					if len(parts) != 5:
						continue
					cls, x_center, y_center, w, h = map(float, parts)

					# Convert YOLO to COCO format
					x = (x_center - w / 2) * width
					y = (y_center - h / 2) * height
					box_width = w * width
					box_height = h * height

					coco_output["annotations"].append({
						"id": annotation_id,
						"image_id": image_id,
						"category_id": category_id,
						"bbox": [x, y, box_width, box_height],
						"area": box_width * box_height,
						"iscrowd": 0
					})
					annotation_id += 1

		image_id += 1

# Save to JSON
with open(coco_path, "w") as f:
	json.dump(coco_output, f, indent=2)

print(f"COCO annotations saved to {coco_path}")

COCO annotations saved to /workspace/dslab25/training/vacuum_pump/coco_annotations.json
